In [1]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "1"
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers
import random
from tensorflow.keras.utils import to_categorical

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

In [19]:
video_url = './videos/samples/'
train_url = f'{video_url}training'
test_url = f'{video_url}test'
anomaly_train_url = f'{train_url}/anomaly'
normal_train_url = f'{train_url}/normal'
anomaly_test_url = f'{test_url}/anomaly'
normal_test_url = f'{test_url}/normal'

In [4]:
classes = ['anomaly','normal']
classes_dict = {'anomaly' : 1, 'normal' : 0}

In [5]:
anomaly = os.listdir(f'{train_url}/anomaly')
print(f'anomaly length : {len(anomaly)}')
normal = os.listdir(f'{train_url}/normal')
print(f'normal length : {len(normal)}')
anomaly_train = anomaly
anomaly_val = [anomaly_train.pop(random.randint(0,len(anomaly_train)-1)) for _ in range(int(0.2*len(anomaly)))]
print(f'anomaly_train length : {len(anomaly_train)}')
print(f'anomaly_val length : {len(anomaly_val)}')
normal_train = normal
normal_val = [normal_train.pop(random.randint(0,len(normal_train)-1)) for _ in range(int(0.2*len(normal)))]
print(f'normal_train length : {len(normal_train)}')
print(f'normal_val length : {len(normal_val)}')

anomaly length : 437
normal length : 437
anomaly_train length : 350
anomaly_val length : 87
normal_train length : 350
normal_val length : 87


In [6]:
print(f'{anomaly_train_url}/{anomaly[0]}')

./videos/samples/training/anomaly/222.mp4


In [20]:
def format_frames(frame, output_size):
    frame = tf.image.convert_image_dtype(frame, tf.float32)
    frame = tf.image.resize_with_pad(frame, *output_size)
    frame = tf.image.random_crop(value=frame, size=(240,240,3))
    return frame

def frames_from_video_file(video_path, n_frames, output_size = (240,320)):
  result = []
  src = cv2.VideoCapture(str(video_path))  
  start = 0
  src.set(cv2.CAP_PROP_POS_FRAMES, start)
  frame_step = int((src.get(cv2.CAP_PROP_FRAME_COUNT) // n_frames) - 1)

  ret, frame = src.read()
  result.append(format_frames(frame, output_size))

  for _ in range(n_frames - 1):
    for _ in range(frame_step):
      ret, frame = src.read()
    if ret:
      frame = format_frames(frame, output_size)
      result.append(frame)
    else:
      result.append(np.zeros_like(result[0]))
  src.release()
  result = np.array(result)[..., [2, 1, 0]]

  return result

class FrameGenerator:
  def __init__(self,anomaly, normal, n_frames,testing = False):
    self.n_frames = n_frames
    self.anomaly = anomaly
    self.normal = normal
    self.testing = testing

  def get_files_and_class_names(self):
    video_paths = []
    labels = []
    for path in self.anomaly:
      if self.testing:
        video_paths.append(f'{anomaly_test_url}/{path}')
      else:
        video_paths.append(f'{anomaly_train_url}/{path}')
      labels.append('anomaly')
    for path in self.normal:
      if self.testing:
        video_paths.append(f'{normal_test_url}/{path}')
      else:
        video_paths.append(f'{normal_train_url}/{path}')
      labels.append('normal')
    pairs = list(zip(video_paths,labels))
    return pairs

  def __call__(self):
    pairs = self.get_files_and_class_names()

    random.shuffle(pairs)

    for path, name in pairs:
      video_frames = frames_from_video_file(path, self.n_frames)
      label = classes_dict[name]
      yield video_frames, label

In [8]:
fg = FrameGenerator(anomaly=anomaly_train,normal=normal_train,n_frames=30)

frames, label = next(fg())

print(f"Shape: {frames.shape}")
print(f"Label: {label}")

Shape: (30, 240, 240, 3)
Label: 0


In [9]:
output_signature = (tf.TensorSpec(shape = (30, 240, 240, 3), dtype = tf.float32),
                    tf.TensorSpec(shape = (), dtype = tf.int16))
train_ds = tf.data.Dataset.from_generator(FrameGenerator(anomaly=anomaly_train,normal=normal_train,n_frames=30),
                                          output_signature = output_signature)
val_ds = tf.data.Dataset.from_generator(FrameGenerator(anomaly=anomaly_val,normal=normal_val,n_frames=30),
                                          output_signature = output_signature)
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).repeat().prefetch(buffer_size = AUTOTUNE)
val_ds = train_ds.cache().shuffle(200).repeat().prefetch(buffer_size = AUTOTUNE)
batchsize = 1
train_ds = train_ds.batch(batch_size=batchsize)
val_ds = val_ds.batch(batch_size=batchsize)

In [10]:
from tensorflow import keras
from keras import layers
from keras import regularizers
from keras.layers import SpatialDropout3D

In [11]:
input_shape = (None,30,240,240,3)
model = keras.Sequential([
    layers.Input((input_shape[1:])),
    layers.Rescaling(scale=255),
    layers.Conv3D(32,(3,3,3), activation="relu",kernel_regularizer=regularizers.L1L2()),
    layers.MaxPooling3D(),
    layers.SpatialDropout3D(0.2),
    layers.Conv3D(32,(3,3,3), activation="relu",kernel_regularizer=regularizers.L1L2()),
    layers.MaxPooling3D(),
    layers.SpatialDropout3D(0.2),
    layers.Conv3D(64,(3,3,3), activation="relu",kernel_regularizer=regularizers.L1L2()),
    layers.MaxPooling3D(),
    layers.Flatten(),
    layers.Dense(1, activation="sigmoid")
])

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)           │ (None, 30, 240, 240,   │             0 │
│                                 │ 3)                     │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d (Conv3D)                 │ (None, 28, 238, 238,   │         2,624 │
│                                 │ 32)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d (MaxPooling3D)    │ (None, 14, 119, 119,   │             0 │
│                                 │ 32)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout3d               │ (None, 14, 119, 119,   │             0 │
│ (SpatialDropout3D)              │ 32)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_1 (Conv3D)               │ (None, 12, 117, 117,   │        27,680 │
│                                 │ 32)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_1 (MaxPooling3D)  │ (None, 6, 58, 58, 32)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout3d_1             │ (None, 6, 58, 58, 32)  │             0 │
│ (SpatialDropout3D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_2 (Conv3D)               │ (None, 4, 56, 56, 64)  │        55,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_2 (MaxPooling3D)  │ (None, 2, 28, 28, 64)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 100352)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │       100,353 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 186,017 (726.63 KB)

 Trainable params: 186,017 (726.63 KB)

 Non-trainable params: 0 (0.00 B)

In [12]:
model.compile(loss = keras.losses.BinaryCrossentropy(), 
              optimizer = keras.optimizers.Adam(), 
              metrics = ['accuracy','precision'])

In [13]:
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

In [14]:
reduce_lr = ReduceLROnPlateau(monitor='val_precision',patience=5)
earlystopping = EarlyStopping(patience=5,monitor="val_precision", restore_best_weights=True)
history = model.fit(x = train_ds,
                    epochs = 20, 
                    validation_data = val_ds,
                    steps_per_epoch=700 //batchsize,
                    validation_steps=174 //batchsize,
                    callbacks=[reduce_lr,earlystopping])

Epoch 1/20
700/700 ━━━━━━━━━━━━━━━━━━━━ 1821s 2s/step - accuracy: 0.7437 - loss: 83.4945 - precision: 0.7353 - val_accuracy: 0.8161 - val_loss: 0.6781 - val_precision: 0.7288 - learning_rate: 0.0010
Epoch 2/20
700/700 ━━━━━━━━━━━━━━━━━━━━ 1354s 2s/step - accuracy: 0.6862 - loss: 20.3467 - precision: 0.6752 - val_accuracy: 0.8506 - val_loss: 0.3904 - val_precision: 0.7615 - learning_rate: 0.0010
Epoch 3/20
700/700 ━━━━━━━━━━━━━━━━━━━━ 1345s 2s/step - accuracy: 0.8500 - loss: 0.6263 - precision: 0.7789 - val_accuracy: 0.8333 - val_loss: 0.4195 - val_precision: 0.7686 - learning_rate: 0.0010
Epoch 4/20
700/700 ━━━━━━━━━━━━━━━━━━━━ 1378s 2s/step - accuracy: 0.8373 - loss: 0.3967 - precision: 0.7746 - val_accuracy: 0.8448 - val_loss: 0.4384 - val_precision: 0.7965 - learning_rate: 0.0010
Epoch 5/20
700/700 ━━━━━━━━━━━━━━━━━━━━ 1362s 2s/step - accuracy: 0.8453 - loss: 0.5745 - precision: 0.7745 - val_accuracy: 0.8678 - val_loss: 0.3562 - val_precision: 0.8019 - learning_rate: 0.0010
Epoch 6/

2024-03-16 10:51:23.633137: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [15]:
anomaly_test = os.listdir(f'{test_url}/anomaly')
print(f'anomaly length : {len(anomaly)}')
normal_test = os.listdir(f'{test_url}/normal')
print(f'normal length : {len(normal)}')

anomaly length : 350
normal length : 350


In [18]:
anomaly_test[0]

'96.mp4'

In [23]:
test_ds = tf.data.Dataset.from_generator(FrameGenerator(anomaly=anomaly_test,normal=normal_test,n_frames=30, testing=True),
                                          output_signature = output_signature)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size = AUTOTUNE)
test_ds = test_ds.batch(batch_size=batchsize)

In [24]:
model.evaluate(test_ds)

98/98 ━━━━━━━━━━━━━━━━━━━━ 56s 281ms/step - accuracy: 0.8495 - loss: 0.4855 - precision: 0.7438


2024-03-16 11:37:26.787489: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


[0.46752288937568665, 0.8469387888908386, 0.7833333611488342]

In [25]:
model.save('camnuvem_anomaly_detection_model.keras')